This file was used to extract and combine poses taken from the video. Since each extraction for each video was done throughout the project, each video was processed one by one. This code was used a utility code to provide poses thats why the code had almost no comments. The code was taken from the notebook that uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git)

## Install OpenPose

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The drive was mounted on the colab to keep a backup file of poses.

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

## Detect poses on a test video

In [ ]:
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/drive/MyDrive/Caitlinwalk.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

visualize the result:

In [ ]:
import io
import base64
from IPython.display import HTML

def show_local_mp4_video(file_name, width=640, height=480):
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/drive/MyDrive/output.mp4', width=960, height=720)

In [333]:
import os,json
import numpy as np
path_to_json = '/content/openpose/output/'
final = []
number = 0
for file_name in [file for file in sorted(os.listdir(path_to_json)) if file.endswith('.json')]:
  with open(path_to_json + file_name) as json_file:
    d = json.load(json_file)
    
    for person in d['people']:
     
      A = []
      A = [person['pose_keypoints_2d']]
      X = np.asarray(A)
      even = np.arange(0, X.shape[1], 3)
      odd = np.arange(1, X.shape[1], 3)
      X1 =  X[:,even]
      X2 = X[:,odd]
      Xch = np.dstack((X1,X2))
      number += 1
      if(len(final) == 0):
        final = Xch
      else:
        final = np.concatenate((final, Xch), axis=0)

    
    if(number == 660): break
                

In [ ]:
print(final.shape)

In [335]:
np.save("walk.npy",final)

In [288]:
# This process was done manually. So, it had to be sure that pose frame had a proper label
knocking = [1.,0.,0.,0.]
lifting = [0.,1.,0.,0.]
throwing = [0.,0.,1.,0.]
walking = [0.,0.,0.,1.]
knocking = np.tile(knocking,(44,60,1))
lifting = np.tile(lifting,(44,60,1))
throwing = np.tile(throwing,(44,60,1))
walking = np.tile(walking,(44,60,1))
y_train = np.vstack((knocking,throwing))
y_train = np.vstack((y_train,lifting))
np.save("labels.npy", y_train)
y_train.shape

(132, 60, 4)